<a href="https://colab.research.google.com/github/Mindauskas/bbc_news/blob/master/bbc_news.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
try:
   %tensorflow_version 2.x
except Exception:
   pass

TensorFlow 2.x selected.


In [0]:
SETUP = True

In [3]:
if SETUP:
    !pip install -q -U toai==0.2.*
    !pip install -q -U nb_black
    !pip install -q -U tensorflow-datasets
    !pip install -q -U --no-deps tensorflow-addons
    !pip install -q -U tensorflow_hub
    !pip install -q -U git+https://github.com/huggingface/transformers
    print(__import__("toai").__version__)
    print(__import__("tensorflow").__version__)

     |████████████████████████████████| 81kB 6.1MB/s 
     |████████████████████████████████| 552kB 14.4MB/s 
     |████████████████████████████████| 296kB 44.1MB/s 
     |████████████████████████████████| 153kB 52.2MB/s 
     |████████████████████████████████| 122kB 43.5MB/s 
     |████████████████████████████████| 61kB 8.6MB/s 
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.7.1 which is incompatible.
ERROR: google-colab 1.0.0 has requirement requests~=2.21.0, but you'll have requests 2.22.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement six~=1.12.0, but you'll have six 1.13.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 102kB 4.6MB/s 
     |████████████████████████████████| 645kB 15.5MB/s 
     |████████████████████████████████| 737kB 57.0MB/s 
     |████████████████████████████████| 1.8MB 5.0MB/s

In [4]:
from toai.imports import *
from toai.data import DataBundle, DataParams, DataContainer
from toai.metrics import sparse_top_2_categorical_accuracy
from toai.models import save_keras_model, load_keras_model
from toai.utils import save_file, load_file
import tensorflow as tf
from tensorflow import keras
import tensorflow_addons as tfa
import tensorflow_datasets as tfds
import tensorflow_hub as hub
import csv
import transformers

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
/usr/local/lib/python3.6/dist-packages/toai/imports.py:70: UserWarning: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.
  warnings.warn(str(error))


In [5]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [0]:
DATA_DIR = Path("data/bbc_news")
DATA_DIR.mkdir(parents=True, exist_ok=True)

TEMP_DIR = Path("temp/bbc_news")
TEMP_DIR.mkdir(parents=True, exist_ok=True)

In [0]:
if SETUP:
    shutil.rmtree(str(DATA_DIR))
    shutil.rmtree(str(TEMP_DIR))
    DATA_DIR.mkdir(parents=True, exist_ok=True)
    TEMP_DIR.mkdir(parents=True, exist_ok=True)
    #shutil.copy("bbc_news.pickle", DATA_DIR)

In [0]:
BATCH_SIZE = 8
AUTOTUNE = tf.data.experimental.AUTOTUNE

# **Dataset**

The datset is taken from https://www.kaggle.com/pariza/bbc-news-summary.
This dataset for extractive text summarization has four hundred and seventeen political news articles of BBC from 2004 to 2005 in the News Articles folder. For each articles, five summaries are provided in the Summaries folder. The first clause of the text of articles is the respective title.

In [0]:
with open("bbc_news.pickle", "rb") as f:
    data = pickle.load(f)

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200853 entries, 0 to 200852
Data columns (total 6 columns):
category             200853 non-null object
headline             200853 non-null object
authors              200853 non-null object
link                 200853 non-null object
short_description    200853 non-null object
date                 200853 non-null datetime64[ns]
dtypes: datetime64[ns](1), object(5)
memory usage: 9.2+ MB


In [11]:
data.describe(include="all")

,category,headline,authors,link,short_description,date
count,200853,200853,200853,200853,200853,200853
unique,41,199344,27993,200812,178353,2309
top,POLITICS,Sunday Roundup,,https://www.huffingtonpost.comhttp://www.mothe...,,2013-01-17 00:00:00
freq,32739,90,36620,2,19712,100
first,NaN,NaN,NaN,NaN,NaN,2012-01-28 00:00:00
last,NaN,NaN,NaN,NaN,NaN,2018-05-26 00:00:00


In [12]:
data.head(5).T

,0,1,2,3,4
category,CRIME,ENTERTAINMENT,ENTERTAINMENT,ENTERTAINMENT,ENTERTAINMENT
headline,There Were 2 Mass Shootings In Texas Last Week...,Will Smith Joins Diplo And Nicky Jam For The 2...,Hugh Grant Marries For The First Time At Age 57,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Julianna Margulies Uses Donald Trump Poop Bags...
authors,Melissa Jeltsen,Andy McDonald,Ron Dicker,Ron Dicker,Ron Dicker
link,https://www.huffingtonpost.com/entry/texas-ama...,https://www.huffingtonpost.com/entry/will-smit...,https://www.huffingtonpost.com/entry/hugh-gran...,https://www.huffingtonpost.com/entry/jim-carre...,https://www.huffingtonpost.com/entry/julianna-...
short_description,She left her husband. He killed their children...,Of course it has a song.,The actor and his longtime girlfriend Anna Ebe...,The actor gives Dems an ass-kicking for not fi...,"The ""Dietland"" actress said using the bags is ..."
date,2018-05-26 00:00:00,2018-05-26 00:00:00,2018-05-26 00:00:00,2018-05-26 00:00:00,2018-05-26 00:00:00


In [0]:
def merge_str(row):
  return row['headline'] + '. ' + row['short_description']

data["merged_text"] = data.apply(merge_str, axis=1)
data = data[['category','merged_text']]

In [14]:
data.iloc[3,:]

category                                           ENTERTAINMENT
merged_text    Jim Carrey Blasts 'Castrato' Adam Schiff And D...
Name: 3, dtype: object

In [15]:
data.category.value_counts()

POLITICS          32739
WELLNESS          17827
ENTERTAINMENT     16058
TRAVEL             9887
STYLE & BEAUTY     9649
PARENTING          8677
HEALTHY LIVING     6694
QUEER VOICES       6314
FOOD & DRINK       6226
BUSINESS           5937
COMEDY             5175
SPORTS             4884
BLACK VOICES       4528
HOME & LIVING      4195
PARENTS            3955
THE WORLDPOST      3664
WEDDINGS           3651
WOMEN              3490
IMPACT             3459
DIVORCE            3426
CRIME              3405
MEDIA              2815
WEIRD NEWS         2670
GREEN              2622
WORLDPOST          2579
RELIGION           2556
STYLE              2254
SCIENCE            2178
WORLD NEWS         2177
TASTE              2096
TECH               2082
MONEY              1707
ARTS               1509
FIFTY              1401
GOOD NEWS          1398
ARTS & CULTURE     1339
ENVIRONMENT        1323
COLLEGE            1144
LATINO VOICES      1129
CULTURE & ARTS     1030
EDUCATION          1004
Name: category, 

In [0]:
def keep_values(df, col_name, values):
    return df.loc[df[col_name].isin(values), :].reset_index(drop=True)

In [0]:
df = keep_values(
    data, "category", data["category"].value_counts()[:15].index
)

In [18]:
df["category"].value_counts()

POLITICS          32739
WELLNESS          17827
ENTERTAINMENT     16058
TRAVEL             9887
STYLE & BEAUTY     9649
PARENTING          8677
HEALTHY LIVING     6694
QUEER VOICES       6314
FOOD & DRINK       6226
BUSINESS           5937
COMEDY             5175
SPORTS             4884
BLACK VOICES       4528
HOME & LIVING      4195
PARENTS            3955
Name: category, dtype: int64

In [0]:
def make_category_map(labels) -> Dict[str, int]:
    return {x: i for i, x in enumerate(sorted(set(labels)))}

In [0]:
def init_category_map(filename, labels):
    try:
        category_map = load_file(filename)
    except:
        category_map = make_category_map(labels)
        save_file(category_map, filename)
    return category_map

In [0]:
category_map = init_category_map(
    TEMP_DIR / "category_map.pickle", df["category"].values
)

In [22]:
category_map

{'BLACK VOICES': 0,
 'BUSINESS': 1,
 'COMEDY': 2,
 'ENTERTAINMENT': 3,
 'FOOD & DRINK': 4,
 'HEALTHY LIVING': 5,
 'HOME & LIVING': 6,
 'PARENTING': 7,
 'PARENTS': 8,
 'POLITICS': 9,
 'QUEER VOICES': 10,
 'SPORTS': 11,
 'STYLE & BEAUTY': 12,
 'TRAVEL': 13,
 'WELLNESS': 14}

In [0]:
n_categories = len(category_map)

In [24]:
n_categories

15

In [0]:
df["category"] = df["category"].map(category_map)

In [26]:
df.category.value_counts()

9     32739
14    17827
3     16058
13     9887
12     9649
7      8677
5      6694
10     6314
4      6226
1      5937
2      5175
11     4884
0      4528
6      4195
8      3955
Name: category, dtype: int64

In [0]:
all_data_bundle = DataBundle.from_dataframe(
    dataframe=df, x_col="merged_text", y_col="category"
)

In [0]:
train_data_bundle, validation_data_bundle, test_data_bundle = DataBundle.split(
    data_bundle=all_data_bundle, fracs=[0.2, 0.2, 0.2]
)

In [0]:
data_container = DataContainer(
    train_data_bundle, validation_data_bundle, test_data_bundle
)

In [30]:
data_container.train.value_counts()

{0: 861,
 1: 1199,
 2: 1038,
 3: 3200,
 4: 1311,
 5: 1314,
 6: 776,
 7: 1713,
 8: 821,
 9: 6561,
 10: 1219,
 11: 1033,
 12: 1895,
 13: 2014,
 14: 3594}

In [0]:
class_weights = dict(
    enumerate(
        sk.utils.class_weight.compute_class_weight(
            "balanced", np.unique(data_container.train.y), data_container.train.y
        )
    )
)

In [0]:
data_container.train = DataBundle.from_unbalanced(data_container.train)

In [33]:
data_container.train.value_counts()

{0: 6027,
 1: 5995,
 2: 6228,
 3: 6400,
 4: 6555,
 5: 5256,
 6: 6208,
 7: 5139,
 8: 5747,
 9: 6561,
 10: 6095,
 11: 6198,
 12: 5685,
 13: 6042,
 14: 3594}

In [34]:
len(data_container.train), len(data_container.validation), len(data_container.test)

(87730, 28549, 28549)

In [0]:
def make_dataset(data_bundle):
    return tf.data.Dataset.from_tensor_slices((data_bundle.x, data_bundle.y))

In [0]:
data_container.train.dataset = make_dataset(data_container.train)
data_container.validation.dataset = make_dataset(data_container.validation)
data_container.test.dataset = make_dataset(data_container.test)

In [37]:
data_container.train.x[5]

"Jamie Foxx's Daughter Corinne Named Miss Golden Globe 2016. Jamie Foxx is one proud papa. \xa0 The actor's 21-year-old daughter Corinne Foxx has been named Miss Golden Globe 2016 by the"

In [38]:
data_container.train.y[5]

0

In [0]:
def make_sentence_length_limiter(limit):
    def inner(x, y):
        return tf.strings.substr(x, 0, limit), y

    return inner

In [0]:
length_limiter = make_sentence_length_limiter(300)

In [0]:
train_dataset = (
    data_container.train.dataset.repeat()
    .shuffle(len(data_container.train))
    .batch(BATCH_SIZE)
    .map(length_limiter)
    .prefetch(AUTOTUNE)
)

validation_dataset = (
    data_container.validation.dataset.batch(BATCH_SIZE)
    .map(length_limiter)
    .prefetch(AUTOTUNE)
)

In [0]:
train_dataset_steps = math.ceil(len(data_container.train) / BATCH_SIZE)

## **Tf hub pretrained models**

In [0]:
def train_model(
    model,
    epochs,
    lrs=None,
    optimizers=None,
    patience=5,
    verbose=1,
    log_dir=str(TEMP_DIR / "logs"),
):
    if optimizers is None:
        optimizers = [keras.optimizers.Adam(lr) for lr in lrs]
    model.layers[0].trainable = False
    model.compile(
        loss=keras.losses.sparse_categorical_crossentropy,
        optimizer=optimizers[0],
        metrics=[
            keras.metrics.sparse_categorical_accuracy,
            sparse_top_2_categorical_accuracy,
        ],
    )
    model.fit(
        train_dataset,
        steps_per_epoch=train_dataset_steps,
        validation_data=validation_dataset,
        epochs=epochs[0],
        callbacks=[
            keras.callbacks.ReduceLROnPlateau(patience=1, factor=0.3),
            keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True),
        ],
        class_weight=class_weights,
        verbose=verbose,
    )
    model.layers[0].trainable = True
    model.compile(
        loss=keras.losses.sparse_categorical_crossentropy,
        optimizer=optimizers[1],
        metrics=[
            keras.metrics.sparse_categorical_accuracy,
            sparse_top_2_categorical_accuracy,
        ],
    )
    model.fit(
        train_dataset,
        steps_per_epoch=train_dataset_steps,
        validation_data=validation_dataset,
        epochs=epochs[1],
        callbacks=[
            keras.callbacks.ReduceLROnPlateau(patience=patience // 2, factor=0.3),
            keras.callbacks.EarlyStopping(patience=patience, restore_best_weights=True),
        ],
        class_weight=class_weights,
        verbose=verbose,
    )

In [0]:
def make_hub_model(url):
    return keras.Sequential(
        [
            hub.KerasLayer(url, dtype=tf.string, input_shape=[]),
            keras.layers.Dropout(0.5),
            keras.layers.Dense(n_categories, activation=keras.activations.softmax),
        ]
    )

In [0]:
def run_models(urls):
    for url in urls:
        model = make_hub_model(url)
        model_name = f"{url.split('/')[4]}"
        print(f" {model_name} ".center(80, "="))
        shutil.rmtree(str(TEMP_DIR / model_name), ignore_errors=True)
        train_model(
            model=model,
            epochs=[8, 8],
            optimizers=[keras.optimizers.Adam(lr=1e-4), keras.optimizers.Adam(lr=3e-5)],
            patience=2,
            verbose=2,
            log_dir=str(TEMP_DIR / model_name),
        )
        model.save(f"{TEMP_DIR / model_name}.h5")
        save_keras_model(
            model,
            str(TEMP_DIR / model_name / "architecture"),
            str(TEMP_DIR / model_name / "weights"),
        )
        keras.backend.clear_session()
        del model
        keras.backend.clear_session()

In [0]:
model_urls = (
    "https://tfhub.dev/google/Wiki-words-250-with-normalization/2",
    "https://tfhub.dev/google/nnlm-en-dim128-with-normalization/2",
    "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
)

In [0]:
run_models(model_urls)

INFO:absl:Using /tmp/tfhub_modules to cache modules.
INFO:absl:Downloading TF-Hub Module 'https://tfhub.dev/google/Wiki-words-250-with-normalization/2'.
INFO:absl:Downloaded https://tfhub.dev/google/Wiki-words-250-with-normalization/2, Total size: 970.91MB
INFO:absl:Downloaded TF-Hub Module 'https://tfhub.dev/google/Wiki-words-250-with-normalization/2'.


====================== Wiki-words-250-with-normalization =======================
Train for 10967 steps, validate for 3569 steps
Epoch 1/8
10967/10967 - 59s - loss: 2.4123 - sparse_categorical_accuracy: 0.2404 - sparse_top_2_categorical_accuracy: 0.3793 - val_loss: 2.2445 - val_sparse_categorical_accuracy: 0.4100 - val_sparse_top_2_categorical_accuracy: 0.5497
Epoch 2/8
10967/10967 - 58s - loss: 2.1196 - sparse_categorical_accuracy: 0.3577 - sparse_top_2_categorical_accuracy: 0.5231 - val_loss: 1.9961 - val_sparse_categorical_accuracy: 0.4472 - val_sparse_top_2_categorical_accuracy: 0.6024
Epoch 3/8
10967/10967 - 57s - loss: 1.9873 - sparse_categorical_accuracy: 0.3964 - sparse_top_2_categorical_accuracy: 0.5636 - val_loss: 1.8808 - val_sparse_categorical_accuracy: 0.4657 - val_sparse_top_2_categorical_accuracy: 0.6258
Epoch 4/8
10967/10967 - 57s - loss: 1.9106 - sparse_categorical_accuracy: 0.4163 - sparse_top_2_categorical_accuracy: 0.5849 - val_loss: 1.7903 - val_sparse_categorical_a

In [0]:
print(
    classification_report(
        data_container.validation.y, 
        load_keras_model(
            str(TEMP_DIR/model_urls[0].split('/')[4]/'architecture'),
            str(TEMP_DIR/model_urls[0].split('/')[4]/'weights'),
            custom_objects={'KerasLayer':hub.KerasLayer}
        ).predict(data_container.validation.x).argmax(axis=1)))


In [0]:
print(
    classification_report(
        data_container.validation.y, 
        load_keras_model(
            str(TEMP_DIR/model_urls[1].split('/')[4]/'architecture'),
            str(TEMP_DIR/model_urls[1].split('/')[4]/'weights'),
            custom_objects={'KerasLayer':hub.KerasLayer}
        ).predict(data_container.validation.x).argmax(axis=1)))


In [0]:
print(
    classification_report(
        data_container.validation.y, 
        load_keras_model(
            str(TEMP_DIR/model_urls[2].split('/')[4]/'architecture'),
            str(TEMP_DIR/model_urls[2].split('/')[4]/'weights'),
            custom_objects={'KerasLayer':hub.KerasLayer}
        ).predict(data_container.validation.x).argmax(axis=1)))


In [0]:
data_container.validation.y

In [0]:
classification_report(load_keras_model(
    str(TEMP_DIR/model_urls[0].split('/')[4]/'architecture'),
    str(TEMP_DIR/model_urls[0].split('/')[4]/'weights'),
    custom_objects={"KerasLayer":hub.KerasLayer}
    ).predict(validation_dataset).argmax(axis=1)


In [0]:
print(
    classification_report(
        data_container.test.y, 
        load_keras_model(
            TEMP_DIR/model_urls[2].split('/')[4]/'architecture',
            TEMP_DIR/model_urls[2].split('/')[4]/'weights.index'
        )
        ).predict(test_dataset).argmax(axis=1))


## **BERT model**

In [0]:
def make_bert_dataset(data_bundle, tokenizer, sequence_length):
    features = tf.data.Dataset.from_tensor_slices(
        [
            np.pad(
                tokenizer.encode(x, add_special_tokens=True),
                (0, sequence_length),
                "constant",
                constant_values=tokenizer.pad_token_id,
            )[:sequence_length]
            for x in data_bundle.x
        ]
    )
    labels = tf.data.Dataset.from_tensor_slices(data_bundle.y)
    dataset = tf.data.Dataset.zip((features, labels)).map(
        lambda x, y: (
            {
                "input_ids": x,
                "attention_mask": int(x != tokenizer.pad_token_id),
                "token_type_ids": int(x >= tokenizer.pad_token_id),
            },
            y,
        ),
        num_parallel_calls=AUTOTUNE,
    )
    return dataset

In [0]:
config = transformers.BertConfig.from_pretrained(
    "bert-base-cased", num_labels=n_categories
)

In [0]:
tokenizer = transformers.BertTokenizer.from_pretrained("bert-base-cased")

In [0]:
train_bert_dataset = (
    make_bert_dataset(data_container.train, tokenizer, 128)
    .repeat()
    .shuffle(len(data_container.train))
    .batch(BATCH_SIZE)
    .prefetch(AUTOTUNE)
)

In [0]:
validation_bert_dataset = (
    make_bert_dataset(data_container.validation, tokenizer, 128)
    .batch(BATCH_SIZE)
    .prefetch(AUTOTUNE)
)

In [0]:
model = transformers.TFBertForSequenceClassification.from_pretrained(
    "bert-base-cased", config=config
)

In [0]:
model.layers[0].trainable = False

In [0]:
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(lr=1e-5),
    metrics=[keras.metrics.SparseCategoricalAccuracy("accuracy")],
)

In [0]:
def train_model_bert(
    model,
    epochs,
    train_dataset,
    train_dataset_steps,
    validation_dataset,
    lrs=None,
    optimizers=None,
    patience=5,
    verbose=1,
    log_dir=str(TEMP_DIR / "bert/logs"),
):
    model.layers[0].trainable = False
    if optimizers is None:
        optimizers = [keras.optimizers.Adam(lr) for lr in lrs]
    model.compile(
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        optimizer=optimizers[0],
        metrics=[
            keras.metrics.sparse_categorical_accuracy,
            sparse_top_2_categorical_accuracy,
        ],
    )
    model.fit(
        train_dataset,
        steps_per_epoch=train_dataset_steps,
        validation_data=validation_dataset,
        epochs=epochs[0],
        callbacks=[
            keras.callbacks.ReduceLROnPlateau(patience=1, factor=0.3),
            keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True),
        ],
        class_weight=class_weights,
        verbose=verbose,
    )
    model.layers[0].trainable = True
    model.compile(
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        optimizer=optimizers[1],
        metrics=[
            keras.metrics.sparse_categorical_accuracy,
            sparse_top_2_categorical_accuracy,
        ],
    )
    model.fit(
        train_dataset,
        steps_per_epoch=train_dataset_steps,
        validation_data=validation_dataset,
        epochs=epochs[1],
        callbacks=[
            keras.callbacks.ReduceLROnPlateau(patience=patience // 2, factor=0.3),
            keras.callbacks.EarlyStopping(patience=patience, restore_best_weights=True),
            keras.callbacks.TensorBoard(log_dir=log_dir),
        ],
        class_weight=class_weights,
        verbose=verbose,
    )

In [0]:
train_model_bert(
    model=model,
    epochs=[8, 8],
    train_dataset=train_bert_dataset,
    train_dataset_steps=train_dataset_steps // 5,
    validation_dataset=validation_bert_dataset,
    optimizers=[keras.optimizers.Adam(lr=1e-5), keras.optimizers.Adam(lr=1e-6)],
    patience=2,
    verbose=2,
)

In [0]:
print(
    classification_report(
        [label.numpy() for _, label in data_container.validation.unbatch()],
        model.predict(data_container.validation).argmax(axis=1),
    )
)